In [8]:
import os
import pandas as pd
from kernels.wrapper import MODELS, KernelModelWrapper
from dataset.ipc2023_learning_domain_info import IPC2023_LEARNING_DOMAINS
from itertools import product
from IPython.display import display, HTML

In [9]:
_LOG_DIR = "logs/train_kernel"

ITERATIONS = [1, 3]

In [10]:


def get_df(domain):
  d = {
    "config": [],
    "mse": [],
    "f1": [],
    "nonzero_weights": [],
  }

  CONFIGS = list(product(MODELS, ITERATIONS))
  for config in CONFIGS:
    model, iterations = config
    log_file = "_".join([model, "llg", "ipc2023-learning-"+domain, "wl", str(iterations)])+".log"
    log_file = _LOG_DIR + "/" + log_file
    
    if not os.path.exists(log_file):
      continue

    stats = {
      "config": "_".join([model, "wl", str(iterations)])
    }

    for line in open(log_file, 'r').readlines():
      toks = line.split()
      if "train_mse" in line:
        stats["mse"] = float(toks[-1])
      elif "train_f1_macro" in line:
        stats["f1"] = float(toks[-1])
      elif "zero_weights" in line:
        weights = int(toks[1].split('/')[1])
        zeros = int(toks[1].split('/')[0])
        stats["nonzero_weights"] = weights - zeros
    
    if len(stats) != len(d):
      continue

    for key in stats:
      d[key].append(stats[key])

  df = pd.DataFrame(d)
  return df

In [11]:
for domain in IPC2023_LEARNING_DOMAINS:
  print(domain)
  df = get_df(domain)
  display(df)

blocksworld


,config,mse,f1,nonzero_weights
0,linear-regression_wl_1,0.09,0.93,6219
1,linear-svr_wl_1,0.32,0.45,6150
2,linear-svr_wl_3,0.06,0.99,419878
3,ridge_wl_1,0.25,0.69,6218
4,ridge_wl_3,0.03,0.99,431584
5,lasso_wl_1,3.74,0.23,6
6,lasso_wl_3,3.74,0.23,7


childsnack


,config,mse,f1,nonzero_weights
0,linear-regression_wl_1,0.10,0.88,1119
1,linear-regression_wl_3,0.86,0.78,55020
2,linear-svr_wl_1,0.11,0.91,1113
3,linear-svr_wl_3,0.10,0.92,54322
4,ridge_wl_1,0.10,0.91,1064
5,ridge_wl_3,0.09,0.92,54884
6,lasso_wl_1,1.46,0.38,4
7,lasso_wl_3,1.46,0.38,7


ferry


...

,config,mse,f1,nonzero_weights
0,linear-regression_wl_1,33.44,0.04,201
1,linear-regression_wl_3,324.30,0.04,16974
2,linear-svr_wl_1,42.05,0.03,201
3,linear-svr_wl_3,40.11,0.04,16937
4,ridge_wl_1,33.46,0.04,162
5,ridge_wl_3,22.06,0.11,16894
6,lasso_wl_1,55.27,0.01,6
7,lasso_wl_3,27.32,0.05,153


spanner


,config,mse,f1,nonzero_weights
0,linear-regression_wl_1,0.67,0.43,546
1,linear-regression_wl_3,6.04,0.31,24606
2,linear-svr_wl_1,0.71,0.49,545
3,linear-svr_wl_3,0.51,0.64,23697
4,ridge_wl_1,0.67,0.43,506
5,ridge_wl_3,0.46,0.62,24518
6,lasso_wl_1,2.07,0.21,3
7,lasso_wl_3,1.83,0.24,4


transport


,config,mse,f1,nonzero_weights
0,linear-regression_wl_1,24.80,0.12,881
1,linear-regression_wl_3,11.09,0.80,261413
2,linear-svr_wl_1,33.52,0.19,878
3,linear-svr_wl_3,0.05,0.96,257365
4,ridge_wl_1,25.04,0.12,826
5,ridge_wl_3,0.03,0.95,261298
6,lasso_wl_1,48.14,0.03,10
7,lasso_wl_3,43.53,0.04,16
